In [1]:
from selenium import webdriver
import requests
import requests_cache
from bs4 import BeautifulSoup
import time
import json
import re
import pandas as pd 
from datetime import datetime, timedelta 
from datetime import date
import urllib.robotparser 
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By

# Set up

In [2]:
# YOUR CODE HERE
def denied(response):
    if "\"incomplete_results\":true" in response.text:
        return False
    else:
        return True
requests_cache.install_cache(expire_after = None, filter_fn = denied)

In [3]:
url = "https://soundcloud.com/search/people?q=*"
service = Service("chromedriver.exe")
#cái này setup theo mấy trang nó chỉ!!
driver1 = webdriver.Chrome(service = service)
driver1.get(url)
soup = BeautifulSoup(driver1.page_source)
cookies_class = soup.find("button",{"id":"onetrust-accept-btn-handler"})["id"]
cookies_click = driver1.find_element(By.XPATH,"//button[@id=\'" + cookies_class + "\']")
cookies_click.click()

driver2 = webdriver.Chrome(service = service)
driver2.get(url)
soup = BeautifulSoup(driver2.page_source)
cookies_class = soup.find("button",{"id":"onetrust-accept-btn-handler"})["id"]
cookies_click = driver2.find_element(By.XPATH,"//button[@id=\'" + cookies_class + "\']")
cookies_click.click()

driver3 = webdriver.Chrome(service = service)
driver3.get(url)
soup = BeautifulSoup(driver3.page_source)
cookies_class = soup.find("button",{"id":"onetrust-accept-btn-handler"})["id"]
cookies_click = driver3.find_element(By.XPATH,"//button[@id=\'" + cookies_class + "\']")
cookies_click.click()

## Hàm "dont_have_playlist" dùng để parse track khi user không có playlist nào. 

In [4]:
def dont_have_playlist(page_source):    
    soup = BeautifulSoup(page_source)
    sound_item = soup.find_all("li",{"class","soundList__item"})

    count_item = 0
    for item in sound_item:
        if count_item == 7:
            break
        count_item += 1
        item_url = item.find("a",{"class","sound__coverArt"})["href"]
#             print(item_url)
        sound_url = "https://soundcloud.com/" + item_url
        req = requests.get(sound_url)
        soup = BeautifulSoup(req.text, "lxml")
        search = soup.find_all("script",{"":""})
        for script in search:
            if "window.__sc_hydration" in script.text:
                search = script
                break

        res = json.loads(search.text[24:-1])
        hydratable_sound = res[6]["data"]
#             print(hydratable_sound)
#         for i in hydratable_sound.keys():
#             if hydratable_sound[i] == "":
#                 hydratable_sound[i] = "None"

        for attr in attributes_track:
            output_track.write(str(hydratable_sound[attr]).replace(",",";"))
            output_track.write(",")
        output_track.write("\n")

## Hàm "write_file_track" dùng để ghi các track đã vào file track.csv

In [5]:
def write_file_track(hydratable_sound):
#         for i in hydratable_sound.keys():
#             if hydratable_sound[i] == "":
#                 hydratable_sound[i] = "None"

        for attr in attributes_track:
            output_track.write(str(hydratable_sound[attr]).replace(",",";"))
            output_track.write(",")
        output_track.write("\n")

# Hàm "write_file_playlist" dùng để ghi các playlist vào file playlist.csv

In [6]:
def write_file_playlist(page_source):    
    soup = BeautifulSoup(page_source)
    sound_item = soup.find_all("li",{"class","soundList__item"})

    count_item = 0
    count_track = 0
    for item in sound_item:
        if count_item == 6:
            break
        count_item += 1
        item_url = item.find("a",{"class","sound__coverArt"})["href"]
#             print(item_url)
        sound_url = "https://soundcloud.com/" + item_url
        req = requests.get(sound_url)
        soup = BeautifulSoup(req.text, "lxml")
        search = soup.find_all("script",{"":""})
        for script in search:
            if "window.__sc_hydration" in script.text:
                search = script
                break
#         print(search)
        res = json.loads(search.text[24:-1])
        hydratable_playlist = res[6]["data"]
#             print(hydratable_playlist)
#         for i in hydratable_playlist.keys():
#             if hydratable_playlist[i] == "":
#                 hydratable_playlist[i] = "None"

        for attr in attributes_playlist:
            if attr == "tracks":
                break
            output_playlist.write(str(hydratable_playlist[attr]).replace(",",";").replace("\n","/"))
            output_playlist.write(",")
            
        trackid_list = ""
        for track in hydratable_playlist["tracks"]:
            trackid_list = trackid_list + str(track["id"]) + ";"
            if len(track) > len(attributes_track):
                count_track += 1
                write_file_track(track)
        output_playlist.write(trackid_list)
        output_playlist.write("\n")
    if count_track <= 3: return False
    else: return True

## Các attrinutes cần được ghi ra ở các file csv

In [7]:
attributes_playlist = ["artwork_url","created_at","duration","embeddable_by"
              ,"genre","id","kind","label_name","last_modified","license"
              ,"likes_count","managed_by_feeds","permalink_url","release_date"
              ,"purchase_title","purchase_url","reposts_count","sharing"
              ,"tag_list","title","user_id","is_album"
              ,"published_at","display_date","track_count","tracks"]

attributes_track = ["artwork_url","commentable","comment_count","created_at","downloadable"
             ,"download_count","duration","full_duration","embeddable_by","genre"
             ,"has_downloads_left","id","kind","label_name","last_modified","license"
             ,"likes_count","permalink_url","playback_count","public","purchase_title"
             ,"purchase_url","release_date","reposts_count","sharing","state","streamable"
             ,"tag_list","title","track_format","user_id","visuals","waveform_url","display_date"
             ,"track_authorization","monetization_model","policy"]

output_playlist = open("playlist.csv","w",encoding = "utf-8")
output_track = open("track.csv","w",encoding = "utf-8")

for attr in attributes_playlist:
    output_playlist.write(attr)
    output_playlist.write(",")
output_playlist.write("\n")

for attr in attributes_track:
    output_track.write(attr)
    output_track.write(",")
output_track.write("\n")


1

## Phần chương trình chính 

In [8]:
users_urls = []
users_urls_file = "good_user.txt"
with open(users_urls_file) as f:
    for line in f:
        users_urls.append(line.replace("\n",""))


step = 0
users_urls.append("")

driver1.get(users_urls[0]+"/sets")
time.sleep(1.5)
prev_url = users_urls[0]
print("total:",len(users_urls))
for url in users_urls:
    try:
        step+=1
        if step % 100==0:
            print("%",step)
#             break
        else:
            if step %10==0:
                print("#",end="")
#                 break

        if step == 1: 
            continue
        Have_playlist = True
        if url=="":
            if i%2==0:
                Have_playlist = write_file_playlist(driver1.page_source)
            else:
                Have_playlist = write_file_playlist(driver2.page_source)
        
        if step%2==0:
            driver2.get(url+"/sets")  # playlist nằm ở "https://soundcloud.com/<username>/sets"
            Have_playlist = write_file_playlist(driver1.page_source)
        else:
            driver1.get(url+"/sets")
            Have_playlist = write_file_playlist(driver2.page_source)
            
        if not Have_playlist:
            driver3.get(prev_url+"/tracks") # track nằm ở "https://soundcloud.com/<username>/tracks"
            time.sleep(0.5)
            dont_have_playlist(driver3.page_source)
            
        prev_url = url
    except:
        continue
    
print("finish")
output_playlist.close()
output_track.close()

total: 2571
#########% 100
#########% 200
#########% 300
#########% 400
#########% 500
#########% 600
#########% 700
#########% 800
#########% 900
#########% 1000
#########% 1100
#########% 1200
#########% 1300
#########% 1400
#########% 1500
#########% 1600
#########% 1700
#########% 1800
#########% 1900
#########% 2000
#########% 2100
#########% 2200
#########% 2300
#########% 2400
#########% 2500
#######finish


### Ở đây, chúng tôi sử dụng 3 driven để tiết kiệm thời gian. Giúp cho trong chương trình không xuất hiện thời gian chết.

### Ở phần chương trình chính, chủ yếu là điều phối sao cho 3 driven hoạt động trơn tru nhất có thể. 